# DeepCOLOR analysis with 10X Visium data

## Import libraries

In [5]:
import torch
import scanpy as sc
import numpy as np
import importlib
from matplotlib import pyplot as plt
import deepcolor
np.random.seed(1)
torch.manual_seed(1)
import anndata
import pandas as pd

## Load data

Load in the scRNA-seq data and the 10X Visium spatial data. To skip the preprocessing and training of the data, move straight to [here](#trained_data)

In [6]:
# Load scRNA-seq data
sc_adata = sc.read_h5ad('data/deepcolor_mouseStSt_visium.h5ad')
# Load Visium data
sp_adata = sc.read_h5ad('data/deepcolor_visium.h5ad')

### Preprocessing

Change the layer names for DeepCOLR and convert the type to float64.

In [3]:
sc_adata.layers['count'] = sc_adata.layers.pop('counts')
sp_adata.layers['count'] = sp_adata.layers.pop('counts')
sp_adata.layers['count'] = sp_adata.layers['count'].astype('float64')

Calculate the log1p for the scRNA-seq data.

In [4]:
sc.pp.log1p(sc_adata)

Remove any noise and only take genes that are expressed in both scRNA-seq and spatial data.

In [13]:
sc_adata = sc_adata[:, sc_adata.layers['count'].toarray().sum(axis=0) > 10]
sp_adata = sp_adata[:, sp_adata.layers['count'].sum(axis=0) > 10]
common_genes = np.intersect1d(sc_adata.var_names, sp_adata.var_names)
sc_adata = sc_adata[:, common_genes]
sp_adata = sp_adata[:, common_genes]

## Estimate the spatial distribution

Train the autoencoder with the data. The resultant scRNA-seq have `map2sp`, the probability of a cell in a spot, and `p_mat`, the colocalization matrix in `obsm`.

In [24]:
importlib.reload(deepcolor)
sc_adata, sp_adata = deepcolor.estimate_spatial_distribution(sc_adata, sp_adata, param_save_path='data/opt_params.pt', first_epoch=500, second_epoch=500, layer_name='count')

Loss: 9824.0263671875
Start first opt
loss at epoch 0 is 7493.03955078125
loss at epoch 10 is 5672.26513671875
loss at epoch 20 is 5369.23095703125
loss at epoch 30 is 5243.7919921875
loss at epoch 40 is 5173.23681640625
loss at epoch 50 is 5123.994140625
loss at epoch 60 is 5086.8466796875
loss at epoch 70 is 5058.38525390625
loss at epoch 80 is 5033.72802734375
loss at epoch 90 is 5014.85009765625
Done first opt
Loss: 4952.63232421875
Start second opt
loss at epoch 0 is 51055.48046875
loss at epoch 10 is 41146.48828125
loss at epoch 20 is 36382.23828125
loss at epoch 30 is 33656.58984375
loss at epoch 40 is 31656.236328125
loss at epoch 50 is 30402.833984375
loss at epoch 60 is 29568.86328125
loss at epoch 70 is 29013.6171875
loss at epoch 80 is 28480.826171875
loss at epoch 90 is 28175.58203125
Done second opt
Loss: 27923.685546875


## Read in the trained data

<a id='trained_data'></a>

Load the preprocessed and raw data instead of the raw data.

In [ ]:
# Run the below to save the trained data to a new file
# sc_adata.write('data/deepcolor_mouseStSt_visium.h5ad', compression='gzip')
# sp_adata.write('data/deepcolor_visium.h5ad', compression='gzip')

In [15]:
# scRNA-seq trained data
sc_adata = sc.read_h5ad('data/deepcolor_mouseStSt_visium.h5ad')
# Spatial trained data
sp_adata = sc.read_h5ad('data/deepcolor_visium.h5ad')

## Calculate proximal cell communications

First load the ligand-target matrix of NicheNet. This matrix is taken from NicheNet v2 instead of DeepCOLOR's matrix.

In [10]:
#! wget -O data/ligand_target_df.csv https://www.dropbox.com/s/2z7ogbks4504iya/ligand_target_df.csv?dl=0
#lt_df = pd.read_csv('data/ligand_target_df.csv', index_col=0)
lt_df = pd.read_csv('data/ligand_target_matrix.csv', index_col=0)

Set KCs, LAM (MoMac1), and central vein and capsule macrophages (MoMac2) as receivers.

The figure below is supposed to show the full result, but as the results did not give any colocalized activity, it is empty.

In [16]:
importlib.reload(deepcolor)
# KCs, MoMac1 & 2
fig, coexp_cc_df = deepcolor.calculate_proximal_cell_communications(sc_adata, 'annot_fine_zonated', lt_df, ["KCs","MoMac1","MoMac2"], celltype_sample_num=500, ntop_genes=4000, each_display_num=3, role="receiver", edge_thresh=1)
fig

c:\Users\yunseolp\anaconda3\envs\test\Lib\site-packages\deepcolor\workflow.py:253: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sc_adata = sc_adata[sc_adata.obs.groupby(celltype_label).sample(celltype_sample_num, replace=True).index]
c:\Users\yunseolp\anaconda3\envs\test\Lib\site-packages\anndata\_core\anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
c:\Users\yunseolp\anaconda3\envs\test\Lib\site-packages\deepcolor\workflow.py:277: ImplicitModificationWarning: Setting element `.layers['activity']` of view, initializing view as actual.
  ligand_adata.layers['activity'] = make_top_values(top_exps @ lt_df)
c:\Users\yunseolp\anaconda3\envs\test\Lib\site-packages\deepcolor\workflow.py:247: FutureWar

There is no result with coactivity score above 0.

In [17]:
coexp_cc_df[(coexp_cc_df['coactivity']>0)].sort_values('coactivity', ascending=False)

,cell1_type,cell2_type,ligand,coactivity
